# Arquivo de Execução para Carregar os Datasets na AWS

* Argumentos:
  * dataset: Nome do(s) dataset(s) que serão carregados separados por ','

Criação dos widgets que receberão os argumentos

In [3]:
dbutils.widgets.text("datasets", "")

Atribuição dos valores dos argumentos às variáveis

In [5]:
datasets = dbutils.widgets.get("datasets")

Importação da classe de DatasetBase

In [7]:
%run ../Datasets/DatasetBase

* Criação do Bucket que armazena os arquivos com os Datasets cadastrados na AWS.
* Leitura dos arquivos com os Datasets cadastrados.
* Exibição dos Datasets cadastrados.

In [9]:
from pyspark.sql.functions import lower, col
datasets = str(datasets).replace(' ','').lower().split(',')

bucket_write = dbutils.secrets.get(scope = "aws", key = "aws-bucket-name")
DatasetBase.createBucket(bucket_write)

complete_path = f'/mnt/{bucket_write}/raw/datasets'

datasetsDataFrame = DatasetBase.loadDatabase(bucket_write, complete_path)
display(datasetsDataFrame)

* Criação dos buckets utilizados
* Leitura do arquivo no source
* Escrita na AWS dos datasets em parquet passados como argumento

Obs.: Para melhorar a performance, poderia ser utilizada uma política de carregamento de Delta dos arquivos

In [11]:
from datetime import datetime

for dataset in datasets:
    if datasetsDataFrame.filter(lower(col('name')) == dataset).count() == 1:
        _, name, bucket_name, path, delimiter, extension, _ = datasetsDataFrame.filter(lower(col('name')) == dataset).collect()[0]
        
        newDataset = DatasetBase(bucket_name, path, delimiter, extension)
        DatasetBase.createBucket(bucket_name)
        
        complete_path = f'/mnt/{bucket_name}/{path}'
        newDataset.setVariables()
        df = newDataset.loadDatabase(bucket_name, complete_path, extension)
        
        path_write = f'/mnt/{bucket_write}/raw/{name.lower()}/{datetime.now().strftime("%Y")}/{datetime.now().strftime("%m")}/{datetime.now().strftime("%d")}'
        DatasetBase.writeDataset(df, bucket_write, path_write)
        
        print(f'Dataset {name} carregado com Sucesso')
    else:
        print(f'Dataset {name} não cadastrado!!')

Exibição dos Arquivos gravados na AWS

In [13]:
display(dbutils.fs.ls(f'/mnt/{bucket_write}/raw/'))